In [16]:
import ktrain
from ktrain import text
import pickle
import os
import keras_bert
import zipfile
from sklearn.datasets import load_files
import chardet

In [7]:
def standardize_to_utf8(encoding):
    """
    standardize to utf-8 if necessary.
    NOTE: mainly used to use utf-8 if ASCII is detected, as
    BERT performance suffers otherwise.
    """
    encoding = 'utf-8' if encoding.lower() in ['ascii', 'utf8', 'utf-8'] else encoding
    return encoding

In [12]:
train_test_names = ["train", "test"]
datadir = "dataset_up"
classes = ["class0", "class1"]
train_str = train_test_names[0]
train_b = load_files(os.path.join(datadir, train_str), shuffle=True, categories=classes)
test_str = train_test_names[1]
test_b = load_files(os.path.join(datadir,  test_str), shuffle=False, categories=classes)
x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

In [13]:
pickle.dump(train_b['filenames'], open("pickles/filenames_mapping.pkl","wb"))
pickle.dump(test_b['filenames'], open("pickles/filename_map_test.pkl","wb"))

In [14]:
len(train_b['data'])

25788

In [17]:
lst = [chardet.detect(doc)['encoding'] for doc in x_train[:32]]
encoding = max(set(lst), key=lst.count)
encoding = standardize_to_utf8(encoding)
x_train = [x.decode(encoding) for x in x_train]
x_test = [x.decode(encoding) for x in x_test]

In [18]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train, y_train, x_test, y_test, 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert')

preprocessing train...


preprocessing test...


In [22]:
print(len(x_train[1]))

25788


In [28]:
print(x_train[1][4])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [74]:
BERT_URL = 'https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip'
# BERT_URL_MULTI = 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip'
# BERT_URL_CN = 'https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip'

In [75]:
def fname_from_url(url):
    return os.path.split(url)[-1]

In [76]:
def get_ktrain_data():
    home = os.path.expanduser('~')
    ktrain_data = os.path.join(home, 'ktrain_data')
    if not os.path.isdir(ktrain_data):
        os.mkdir(ktrain_data)
    return ktrain_data

In [77]:
def download(url, filename):
    with open(filename, 'wb') as f:
        response = requests.get(url, stream=True,  verify=False)
        total = response.headers.get('content-length')

        if total is None:
            f.write(response.content)
        else:
            downloaded = 0
            total = int(total)
            for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
                downloaded += len(data)
                f.write(data)
                done = int(50*downloaded/total)
                sys.stdout.write('\r[{}{}]'.format('█' * done, '.' * (50-done)))
                sys.stdout.flush()

In [78]:
def get_bert_path(lang='en'):
    if lang == 'en':
        bert_url = BERT_URL
    ktrain_data = get_ktrain_data()
    zip_fpath = os.path.join(ktrain_data, fname_from_url(bert_url))
    bert_path = os.path.join( ktrain_data, os.path.splitext(fname_from_url(bert_url))[0] )
    if not os.path.isdir(bert_path) or \
       not os.path.isfile(os.path.join(bert_path, 'bert_config.json')) or\
       not os.path.isfile(os.path.join(bert_path, 'bert_model.ckpt.data-00000-of-00001')) or\
       not os.path.isfile(os.path.join(bert_path, 'bert_model.ckpt.index')) or\
       not os.path.isfile(os.path.join(bert_path, 'bert_model.ckpt.meta')) or\
       not os.path.isfile(os.path.join(bert_path, 'vocab.txt')):
        # download zip
        print('downloading pretrained BERT model (%s)...' % (fname_from_url(bert_url)))
        download(bert_url, zip_fpath)

        # unzip
        print('\nextracting pretrained BERT model...')
        with zipfile.ZipFile(zip_fpath, 'r') as zip_ref:
            zip_ref.extractall(ktrain_data)
        print('done.\n')

        # cleanup
        print('cleanup downloaded zip...')
        try:
            os.remove(zip_fpath)
            print('done.\n')
        except OSError:
            print('failed to cleanup/remove %s' % (zip_fpath))
    return bert_path

In [80]:
# # max_features = preproc.max_features if preproc is not None else None
# features = set()
maxlen = 500
lang = 'en'
if lang is None: raise ValueError('lang is missing')
config_path = os.path.join(get_bert_path(lang=lang), 'bert_config.json')
checkpoint_path = os.path.join(get_bert_path(lang=lang), 'bert_model.ckpt')

In [82]:
model = keras_bert.load_trained_model_from_checkpoint(
                                config_path,
                                checkpoint_path,
                                training=True,
                                trainable=True,
                                seq_len=maxlen)

In [101]:
Dense = keras.layers.Dense
Model = keras.Model

In [102]:
inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output

In [103]:
output = Dense(units=256, activation='softmax')(dense)
outputs = Dense(units=2, activation='softmax')(output)
model = Model(inputs, outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['f1score'])

AttributeError: 'tuple' object has no attribute 'layer'

In [ ]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)

In [ ]:
pickle.dump(learner, open("pickles/learner_15.pkl","wb"))